# Example usage - Command line

Example with 4D-Lung sample which uses a RTSTRUCT segmentation

In [ ]:
!readii ../tests/4D-Lung ../tests/output/notebook/ --roi_names Tumor_c.\* --pyradiomics_setting ../src/readii/data/default_pyradiomics.yaml --update true

Example with NSCLC Radiogenomics sample which uses a DICOM-SEG segmentation

In [ ]:
!readii ../tests/NSCLC_Radiogenomics ../tests/output/notebook/ --pyradiomics_setting ../src/readii/data/default_pyradiomics.yaml --update true

# Example usage - Functions

Here we will demonstrate how to use the helper functions in readii to load and preprocess a CT image and a corresponding segmentation in DICOM SEG format.

## Imports

In [ ]:
from readii.loaders import loadDicomSITK, loadSegmentation
from readii.image_processing import flattenImage, alignImages, displayImageSlice, displayCTSegOverlay

## Load a CT and segmentation

We'll first load in a CT image and segmentation to process.

In [ ]:
ctImage = loadDicomSITK("../tests/NSCLC_Radiogenomics/R01-001/09-06-1990-NA-CT_CHEST_ABD_PELVIS_WITH_CON-98785/3.000000-THORAX_1.0_B45f-95741")

In [ ]:
segImages = loadSegmentation("../tests/NSCLC_Radiogenomics/R01-001/09-06-1990-NA-CT_CHEST_ABD_PELVIS_WITH_CON-98785/1000.000000-3D_Slicer_segmentation_result-67652/1-1.dcm",
                            modality = 'SEG')
print(segImages)

We can see here that the segmentation file contained one region of interest labelled 'Heart'.

## Pre-process the segmentation

We can then process the segmentation image so it's ready for analysis with the CT. We process each region of interest (ROI) individually, so we extract the 'Heart' segmentation first.

In [ ]:
roiImage = segImages['Heart']

Now, let's ensure the dimensions match between the CT and heart segmentation.

In [ ]:
print("CT dimensions: ", ctImage.GetSize())
print("Segmentation dimensions: ", roiImage.GetSize())

We can see that the segmentation has an extra dimension with a length of 1. Let's remove this.

In [ ]:
flattenedROIImage = flattenImage(roiImage)
print("Segmentation dimensions: ", flattenedROIImage.GetSize())

Next is aligning the segmentation to the CT.

In [ ]:
alignedROIImage = alignImages(ctImage, flattenedROIImage)

## Plot CT image

We can now plot the CT image.

In [ ]:
displayImageSlice(ctImage, 240)

In [ ]:
displayCTSegOverlay(ctImage, alignedROIImage)

## Extract Radiomic Features

In [ ]:
from readii.feature_extraction import singleRadiomicFeatureExtraction

pyradiomicsParamFilePath = "../src/readii/data/default_pyradiomics.yaml"
radiomicFeatures = singleRadiomicFeatureExtraction(ctImage, roiImage, pyradiomicsParamFilePath)

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(radiomicFeatures, orient="index").transpose()

## Negative Control Creation

In [ ]:
from readii.negative_controls import *
import numpy as np

### Shuffle whole image

In [ ]:
shuffledImage = shuffleImage(ctImage)
originalPixels = sitk.GetArrayFromImage(ctImage)
shuffledPixels = sitk.GetArrayFromImage(shuffledImage)

In [ ]:
displayCTSegOverlay(shuffledImage, alignedROIImage, alpha=0.4)

In [ ]:
from readii.image_processing import getROICenterCoords

getROICenterCoords(alignedROIImage)


In [ ]:
sliceToCheck = int(originalPixels.shape[0] / 2)
columnIdx = int(originalPixels.shape[1] / 2)
rowIdx = columnIdx

In [ ]:
originalColumn = originalPixels[sliceToCheck, columnIdx, :]
originalRow = originalPixels[sliceToCheck, :, rowIdx]

shuffledColumn = shuffledPixels[sliceToCheck, columnIdx, :]
shuffledRow = shuffledPixels[sliceToCheck, :, rowIdx]

In [ ]:
all(originalColumn == shuffledColumn)

In [ ]:
all(originalRow == shuffledRow)

In [ ]:
sum(originalColumn == originalColumn)

### Display negative control